In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from evodiff.pretrained import OA_DM_38M, OA_DM_640M
from evodiff.generate import generate_oaardm
from evodiff.conditional_generation import inpaint_simple 

torch.hub.set_dir('/data/users/kgeorge/workspace/evodiff')

/nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /nethome/kgeorge/miniconda3/envs/workspace-esm/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
from DomainPrediction import BaseProtein
from DomainPrediction.eval import metrics
from DomainPrediction.utils import helper
from DomainPrediction.utils.constants import *

In [4]:
protein = BaseProtein(file='../../Data/gxps/gxps_ATC_hm_6mfy.pdb')

In [5]:
protein.get_residues(T_gxps_atc) ## T domain

'GEIEIALATIWRELLNVEQVGRHDSFFALGGHSLLAVRMIERLRRIGLGLSVQTLFQHPTLSVLAQSLVP'

In [6]:
start_idx, end_idx = A_gxps_atc[-1]+1, C_gxps_atc[0]
start_idx, end_idx

(489, 604)

In [7]:
idr_length = end_idx - start_idx
masked_sequence = protein.sequence[0:start_idx] + '#' * idr_length + protein.sequence[end_idx:]

In [8]:
masked_sequence.count('#') == 115

True

In [9]:
## A and C domains match
masked_sequence

'VCVHQLFEQQIEKTPDAIAVIYENQTLSYAELNARANRLAHQLIALGVAPDQRVAICVTRSLARIIGLLAVLKAGGAYVPLDPAYPGERLAYMLTDATPVILMADNVGRAALSEDILATLTVLDPNTLLEQPDHNPQVSGLTPQHLAYVIYTSGSTGRPKGVMIEHRSVVNLTLTQITQFDVCATSRMLQFASFGFDASVWEIMMALSCGAMLVIPTETVRQDPQRLWRYLEEQAITHACLTPAMFHDGTDLPAIAIKPTLIFAGEAPSPALFQALCSRADLFNAYGPTEITVCATTWDCPADYTGGVIPIGSPVANKRLYLLDEHRQPVPLGTVGELYIGGVGVARGYLNRPELTAERFLNDPFSDETNARMYRAGDLARYLPDGNLVFVGRNDQQVKIRGFRIEPGEIEARLVEHSEVSEALVLALGDGQDKRLVAYVVALADDGLATKLREHLSDILPDYMIPAAFVRLDAFPLTPNGKLDRRSLP###################################################################################################################QAEIDRIVEQVPGGIANIQDIYALSPLQDGILFHHLLANEGDPYLLITQQAFADRPLLNRYLAAVQQVVDRHDILRTAFIWEGLSVPAQVICRQAPLSVTELTLNPADGAISNQLAQRFDPRRHRIDLNQAPLLRFVVAQESDGRWILLQLLHHLIGDHTTLEVMNSEVQACLLGQMDSLPAPVPFRHLVAQARQGVSQAEHTRFFTDMLAEVDEPTLLFGLAEAHHDGSQVTESHRMLTAGLNERLRGQARRLGVSVAALCHLAWAQVLSRTSGQTQVVFGTVLFGRMQAGEGSDSGMGLFINTLPLRLDIDNTPVRDSVRAAHSRLAGLLEHEHASLALAQRCSGVESGTPLFNALLNYRHNTQPVTPDEIVSGIEFLGAQERTNYPFVLSVE

In [10]:
# checkpoint = OA_DM_38M()
checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint

In [11]:
model = model.cuda()

In [12]:
sequence = protein.sequence
start_idx, end_idx = A_gxps_atc[-1]+1, C_gxps_atc[0]

fasta_file = '../../Data/evodiff_experiments/mi_exp/evodiff_1000.fasta'
for i in range(1000):
    seq_dict = {}
    sample, entire_sequence, generated_idr = inpaint_simple(model, sequence, start_idx, end_idx, tokenizer=tokenizer, device='cuda')
    id = f'gxps_ATC_evodiff_gen_{i}'
    seq_dict[id] = entire_sequence

    helper.create_fasta(seq_dict, fasta_file, append=True)


 63%|██████▎   | 72/115 [00:22<00:13,  3.22it/s]